## DATA 620 

---
### Project 1 - Alt
### Mael Illien and Jeremy OBrien

In [1]:
import foursquare
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from config import foursquare_config as fs_config

In [2]:
midtown = pd.read_csv('venues_midtown_unique.csv')
downtown = pd.read_csv('venues_downtown.csv')

In [3]:
# Cleanup datasets
midtown = midtown.filter(items=['id', 'name','type'])
downtown = downtown.drop_duplicates()
downtown = downtown.filter(items=['id', 'name','type'])

In [4]:
print(len(midtown))
midtown.head(10)

21


,id,name,type
0,43695300f964a5208c291fe3,Empire State Building,Building
1,4bcca12bb6c49c7422169491,86th Floor Observation Deck,Scenic Lookout
2,49b7ed6df964a52030531fe3,Times Square,Plaza
3,41102700f964a520d60b1fe3,Macy's,Department Store
4,40b68100f964a5207d001fe3,Madison Square Park,Park
5,3fd66200f964a520d7f11ee3,Bryant Park,Park
6,50b6ed46e4b0a40f1ec2ab32,102nd Floor Observatory,Scenic Lookout
7,4a065273f964a520ec721fe3,M&M's World,Candy Store
8,428a8580f964a52083231fe3,Hard Rock Cafe,Theme Restaurant
9,49b79f54f964a5202c531fe3,Rockefeller Center,Plaza


In [5]:
print(len(downtown))
downtown.head(10)

23


,id,name,type
0,4abe7f9ef964a520068e20e3,One World Trade Center,Building
1,4b6b5abff964a520fb022ce3,National September 11 Memorial & Museum (Natio...,Memorial Site
2,515abd65e4b08eda86b30e6e,One World Observatory,Scenic Lookout
3,49ff62f9f964a52013701fe3,Century 21 Department Store,Department Store
4,4a43bcb7f964a520bba61fe3,Brooklyn Bridge,Bridge
5,4a6fa0abf964a520a4d61fe3,Brookfield Place (BFPL),Mall
9,42daf100f964a52035261fe3,Battery Park,Park
16,55edd0ef498e4b104d071a87,One Dine,New American
17,5984eeb39be522744003dee5,Oculus Plaza,Plaza
20,4b7de017f964a52049d82fe3,Starbucks,Coffee Shop


In [15]:
midtown['id'][0]

'43695300f964a5208c291fe3'

In [7]:
midtown[midtown['id'] == '43695300f964a5208c291fe3']

,id,name,type
0,43695300f964a5208c291fe3,Empire State Building,Building


In [8]:
# Fill these out to run but don't save to Github
CLIENT_ID = fs_config['id']
CLIENT_SECRET = fs_config['secret']

client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [9]:
# We can alternatively use the venue attribute City
bbox = [40.4959929,-74.257159,40.915568,-73.699215]  # bounding box for New York City

In [10]:
# No longer needed
def is_in_NYC(lat, long):
    print((lat > bbox[0]) & (lat < bbox[2]) & (long > bbox[1]) & (long < bbox[3]))
    return (lat > bbox[0]) & (lat < bbox[2]) & (long > bbox[1]) & (long < bbox[3])

In [11]:
# No longer needed
def extract_venue_data2(venue):
    #print(venue)
    
    data = {'id': [venue['venue']['id']],
            'name': [venue['venue']['name']],
            'type': [venue['venue']['categories'][0]['shortName']],
            'lat': [venue['venue']['location']['lat']],
            'lng': [venue['venue']['location']['lng']]}
    return data
    
def extract_venue_data(venue_id, data):
    #print(venue)
    return data[midtown['id'] == venue_id]

In [12]:
def generate_links(locations):
    links = []

    for l in locations['id']:
        #print(l)
        next_locations = client.venues.nextvenues(l)['nextVenues']['items']
        
        for nl in next_locations:
            link = (l, nl['id'])
            #print(link)
            if not link in links:
                #if nl['id'] in locations['id']:
                    #print('adding link')
                links.append(link)          
    return links

def generate_network(locations):
    #labels = dict(zip(list(venues['id']),list(venues['name'])))
    links = generate_links(locations)
    print(links)
    
    G = nx.DiGraph()
    for f, t in links:
        G.add_edge(f, t)
    return G

In [13]:
midtown_g = generate_network(midtown)
print(nx.info(midtown_g))

Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5eea8564aba297001b372145'}
Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5eea875f1187ee001bce12a5'}
Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5eea86f583525f001b24a854'}


FoursquareException: Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5eea86f583525f001b24a854'}

In [ ]:
generate_links(midtown)

In [ ]:
downtown_g = generate_network(downtown)
print(nx.info(downtown_g))

In [ ]:
# Make figures bigger
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
labels = dict(zip(list(midtown['id']),list(midtown['name'])))

In [ ]:
nx.draw(midtown_g, with_labels=True)

In [ ]:
venues = midtown

degree = nx.degree(midtown_g)
venues['degree'] = [degree[n] for n in venues['id']]

pagerank = nx.pagerank(midtown_g, alpha=.9)
venues['pagerank'] = [pagerank[n] for n in venues['id']]

betweenness = nx.betweenness_centrality(midtown_g)
venues['betweenness'] = [betweenness[n] for n in venues['id']]

venues

In [ ]:
graph_pos = nx.spring_layout(midtown_g)
#graph_pos

In [ ]:
# Plot network

#fig = plt.figure(figsize(16,9), dpi=150)
nodesize = [10000 * n for n in pagerank.values()]
nx.draw_networkx_nodes(midtown_g, graph_pos, node_size=nodesize, alpha=.5, node_color='blue')
nx.draw_networkx_edges(midtown_g, graph_pos, width=1, alpha=.3, edge_color='blue')
nx.draw_networkx_labels(midtown_g, graph_pos, labels=labels, font_size=10, font_family='Arial')